<a href="https://colab.research.google.com/github/juanepstein99/DI_Bootcamp/blob/main/Week9/Day3%264/ExercisesXP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Dataset**

In these exercises, we're going to experiment with the Chinook sample DB while using SQLAlchemy module

In [6]:
!wget -q https://github.com/lerocha/chinook-database/releases/download/v1.4.5/Chinook_Sqlite.sqlite -O chinook.db
!ls -l chinook.db

-rw-r--r-- 1 root root 1067008 Feb 12  2024 chinook.db


The helper methods below will help, you may use for the following exercises :

In [7]:
### useful: functions for displaying results from sql queries using pandas
from IPython.display import display
import pandas as pd

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    sql(query)

**🌟 Exercise 1 : Open the database**

* Open the database using sqlalchemy module interface. create an engine object in a variable named engine
* Call the connect() method to obtain a connection and place in a variable named cur

now run the code below to to run reflecton on the database, prepare classes that map to the database and create an orm session :

In [8]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker

engine = create_engine("sqlite:///chinook.db")

cur = engine.connect()

metadata = sqlalchemy.MetaData()
metadata.reflect(engine)

Base = automap_base(metadata=metadata)
Base.prepare()

Session = sessionmaker(bind=engine)
session = Session()

print("Conectado correctamente")

Conectado correctamente


**🌟 Exercise 2 : table names**
* Print out all the table names


In [10]:
print("Tables:")
for t in metadata.tables.keys():
    print("-", t)

Tables:
- Album
- Artist
- Customer
- Employee
- Genre
- Invoice
- InvoiceLine
- Track
- MediaType
- Playlist
- PlaylistTrack


**🌟 Exercise 3 : Tracks**

* Print out the first three tracks in the tracks table

In [12]:
Track = Base.classes.Track

q = session.query(Track.TrackId, Track.Name).order_by(Track.TrackId).limit(3)
for row in q.all():
    print(row)

(1, 'For Those About To Rock (We Salute You)')
(2, 'Balls to the Wall')
(3, 'Fast As a Shark')


**🌟 Exercise 4 : Albums from Tracks**

* Print out the track name and albums title of the first 20 tracks in the tracks table

In [13]:
Album = Base.classes.Album
Track = Base.classes.Track

q = (
    session.query(
        Track.Name.label("track_name"),
        Album.Title.label("album_title")
    )
    .join(Album, Track.AlbumId == Album.AlbumId)
    .order_by(Track.TrackId)
    .limit(20)
)

for row in q.all():
    print(row)

('For Those About To Rock (We Salute You)', 'For Those About To Rock We Salute You')
('Balls to the Wall', 'Balls to the Wall')
('Fast As a Shark', 'Restless and Wild')
('Restless and Wild', 'Restless and Wild')
('Princess of the Dawn', 'Restless and Wild')
('Put The Finger On You', 'For Those About To Rock We Salute You')
("Let's Get It Up", 'For Those About To Rock We Salute You')
('Inject The Venom', 'For Those About To Rock We Salute You')
('Snowballed', 'For Those About To Rock We Salute You')
('Evil Walks', 'For Those About To Rock We Salute You')
('C.O.D.', 'For Those About To Rock We Salute You')
('Breaking The Rules', 'For Those About To Rock We Salute You')
('Night Of The Long Knives', 'For Those About To Rock We Salute You')
('Spellbound', 'For Those About To Rock We Salute You')
('Go Down', 'Let There Be Rock')
('Dog Eat Dog', 'Let There Be Rock')
('Let There Be Rock', 'Let There Be Rock')
('Bad Boy Boogie', 'Let There Be Rock')
('Problem Child', 'Let There Be Rock')
('Over

**🌟 Exercise 5 : Tracks sold**

* Print out the first 10 track sales from the invoice_items table
* For these first 10 sales, print what are the names of the track sold, and the quantity sold

In [16]:
InvoiceLine = Base.classes.InvoiceLine
Track = Base.classes.Track

q = (
    session.query(
        InvoiceLine.InvoiceLineId,
        InvoiceLine.InvoiceId,
        Track.Name.label("track_name"),
        InvoiceLine.Quantity
    )
    .join(Track, InvoiceLine.TrackId == Track.TrackId)
    .order_by(InvoiceLine.InvoiceLineId)
    .limit(10)
)

for row in q.all():
    print(row)

(1, 1, 'Balls to the Wall', 1)
(2, 1, 'Restless and Wild', 1)
(3, 2, 'Put The Finger On You', 1)
(4, 2, 'Inject The Venom', 1)
(5, 2, 'Evil Walks', 1)
(6, 2, 'Breaking The Rules', 1)
(7, 3, 'Dog Eat Dog', 1)
(8, 3, 'Overdose', 1)
(9, 3, 'Love In An Elevator', 1)
(10, 3, "Janie's Got A Gun", 1)


**🌟 Exercise 6 : Top tracks sold**

* Print the names of top 10 tracks sold, and how many they times they were sold

In [17]:
from sqlalchemy import func

InvoiceLine = Base.classes.InvoiceLine
Track = Base.classes.Track

q = (
    session.query(
        Track.Name.label("track_name"),
        func.sum(InvoiceLine.Quantity).label("total_units_sold"),
        func.count(InvoiceLine.InvoiceLineId).label("times_sold")
    )
    .join(Track, InvoiceLine.TrackId == Track.TrackId)
    .group_by(Track.TrackId, Track.Name)
    .order_by(func.sum(InvoiceLine.Quantity).desc())
    .limit(10)
)

for row in q.all():
    print(row)

('Balls to the Wall', 2, 2)
('Inject The Venom', 2, 2)
('Snowballed', 2, 2)
('Overdose', 2, 2)
('Deuces Are Wild', 2, 2)
('Not The Doctor', 2, 2)
('Por Causa De Você', 2, 2)
('Welcome Home (Sanitarium)', 2, 2)
('Snowblind', 2, 2)
('Cornucopia', 2, 2)


**🌟 Exercise 7 : Top selling artists**

* Who are the top 10 highest selling artists?

In [18]:
from sqlalchemy import func

Artist = Base.classes.Artist
Album  = Base.classes.Album
Track  = Base.classes.Track
InvoiceLine = Base.classes.InvoiceLine

q = (
    session.query(
        Artist.Name.label("artist"),
        func.sum(InvoiceLine.UnitPrice * InvoiceLine.Quantity).label("revenue")
    )
    .join(Album, Artist.ArtistId == Album.ArtistId)
    .join(Track, Track.AlbumId == Album.AlbumId)
    .join(InvoiceLine, InvoiceLine.TrackId == Track.TrackId)
    .group_by(Artist.ArtistId, Artist.Name)
    .order_by(func.sum(InvoiceLine.UnitPrice * InvoiceLine.Quantity).desc())
    .limit(10)
)

for row in q.all():
    print(row)

('Iron Maiden', Decimal('138.6000000000'))
('U2', Decimal('105.9300000000'))
('Metallica', Decimal('90.0900000000'))
('Led Zeppelin', Decimal('86.1300000000'))
('Lost', Decimal('81.5900000000'))
('The Office', Decimal('49.7500000000'))
('Os Paralamas Do Sucesso', Decimal('44.5500000000'))
('Deep Purple', Decimal('43.5600000000'))
('Faith No More', Decimal('41.5800000000'))
('Eric Clapton', Decimal('39.6000000000'))
